In [ ]:
# Célula 1 — preparação do ambiente e fórmula analítica (exemplo)
import numpy as np
import pandas as pd
import os
# detectar Colab
IN_COLAB = False
try:
    import google.colab
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    print('Executando no Colab — se desejar salvar modelos monte o Drive: from google.colab import drive; drive.mount(...)')

# Parâmetros de exemplo para demonstração
s_0 = 0.0
m = 5.0  # kg
k = 0.5  # coeficiente de arrasto (modelo linear usado aqui)
t = 5.0  # segundos
g = 9.81  # aceleração da gravidade (m/s^2)

# Fórmulas analíticas (modelo com arrasto linear)
v_analitica = (m * g / k) * (1 - np.exp(-k * t / m))
y_analitica = (m * g / k) * t - (m**2 * g / k**2) * (1 - np.exp(-k * t / m))

print(f'Velocidade analítica: {v_analitica:.4f} m/s')
print(f'Posição analítica: {y_analitica:.4f} m')

Velocidade analítica: 38.60 m/s
Posição analítica: 104.51 m


### 1) Preparação do ambiente

Propósito: importar bibliotecas e detectar ambiente (Colab vs local). Esta célula prepara o runtime evitando reinstalações desnecessárias de pacotes que podem causar conflitos no Colab.

Entradas: nenhuma.

Saídas: módulos importados no namespace (numpy, tensorflow, matplotlib, etc.) e mensagens informativas.

Dicas:
- Não reinstale TensorFlow/numpy no Colab — use o runtime predefinido.
- Se ocorrer erro de import, verifique a versão do Python e do TensorFlow na máquina.


In [ ]:
# Célula 2 — gerar dataset de exemplos (modelo analítico com arrasto linear)
# Definindo constantes
g = 9.81  # aceleração da gravidade (m/s^2)

# Geração de dados
np.random.seed(42)
n_samples = 1000

massas = np.random.uniform(1.0, 10.0, n_samples)  # massa entre 1 e 10 kg
k_arrastos = np.random.uniform(0.1, 1.0, n_samples)  # constante de arrasto entre 0.1 e 1
tempos = np.random.uniform(0.0, 10.0, n_samples)  # tempo entre 0 e 10 segundos

velocidades = []
posicoes = []

for m_val, k_val, t_val in zip(massas, k_arrastos, tempos):
    v_t = (m_val * g / k_val) * (1 - np.exp(-k_val * t_val / m_val))
    y_t = (m_val * g / k_val) * t_val - (m_val**2 * g / k_val**2) * (1 - np.exp(-k_val * t_val / m_val))
    velocidades.append(v_t)
    posicoes.append(y_t)

# Criação do DataFrame
df = pd.DataFrame({
    'massa': massas,
    'constante_de_arrasto': k_arrastos,
    'tempo': tempos,
    'velocidade': velocidades,
    'posicao': posicoes
})
# Salvar em um arquivo CSV local
os.makedirs('data', exist_ok=True)
csv_path = 'data/queda_objeto.csv'
df.to_csv(csv_path, index=False)
print('Dataset salvo em', csv_path)
df.head()

ValueError: not enough values to unpack (expected 4, got 3)

### 2) Definição de funções físicas e utilitários

Propósito: declarar funções reutilizáveis como a solução analítica para arrasto linear, o integrador RK4 para arrasto quadrático, e utilitários para salvar/carregar dados.

Entradas: parâmetros físicos (massa, coef. de arrasto, área, densidade), condições iniciais e vetor de tempo.

Saídas: funções que retornam s(t) e v(t) e funções auxiliares para gerar datasets.

Dicas:
- Verifique assinaturas das funções: mantenha consistência entre m,k,t e outras variáveis.
- Teste as funções individualmente com casos simples (por exemplo, sem arrasto) para validar comportamento.


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os

# Carregar os dados do CSV (caminho atualizado)
csv_path = 'data/queda_objeto.csv'
if not os.path.exists(csv_path):
    raise SystemExit(f'Arquivo de dados não encontrado: {csv_path} - execute a célula de geração de dados primeiro')
df = pd.read_csv(csv_path)

# Separar as entradas (X) e as saídas (y)
X = df[['massa', 'constante_de_arrasto', 'tempo']]
y = df[['velocidade', 'posicao']]

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Padronizar os dados
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

# salvar scalers para inferência posterior
os.makedirs('model', exist_ok=True)
import joblib
joblib.dump(scaler_X, 'model/scaler_X.pkl')
joblib.dump(scaler_y, 'model/scaler_y.pkl')
print('Scalers salvos em model/')

### 3) Loop de geração de amostras / dataset

Propósito: percorrer espaços de parâmetros (massa, arrasto, tempo) para gerar entradas X e saídas y. Pode incluir ruído de medição e escolha entre modelos (analítico vs numérico).

Entradas: intervalos/amostras para m, k (ou Cd/A/rho), t, flags para arrasto e ruído.

Saídas: arrays `X` e `y`, e CSV `data/queda_objeto.csv` salvo.

Dicas:
- Certifique-se de que a ordem das colunas em X seja consistente. Use um DataFrame nomeado se for salvar scalers com nomes de coluna.
- Controle aleatoriedade com um seed para reprodutibilidade.


In [ ]:
# Construir a rede neural
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dense(64, activation='relu'),
    Dense(2, activation='linear')  # Saída de 2 dimensões: velocidade e posição
])

# Compilar o modelo
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

c:\GitHub\ProjetosPessoais\redeNeuralArtificial\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### 4) Pré-processamento e split treino/teste

Propósito: transformar X e y com `StandardScaler`, dividir em treino/teste e salvar os scalers.

Entradas: arrays X, y gerados na célula anterior.

Saídas: `X_train, X_test, y_train, y_test` padronizados; arquivos `model/scaler_X.pkl` e `model/scaler_y.pkl`.

Dicas:
- Ajuste scalers apenas no conjunto de treino para evitar vazamento (data leakage).
- Salve os scalers com joblib para inferência futura.


In [ ]:
# Treinar o modelo com callbacks (salvar melhor modelo)
checkpoint_cb = ModelCheckpoint('model/queda_objeto_best.keras', save_best_only=True, monitor='val_loss')
earlystop_cb = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.2, callbacks=[checkpoint_cb, earlystop_cb])

# salvar o modelo final
model.save('model/queda_objeto.keras')
print('Modelo salvo em model/queda_objeto.keras')

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.7815 - mae: 0.7172 - val_loss: 0.3638 - val_mae: 0.4854
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2932 - mae: 0.4107 - val_loss: 0.0971 - val_mae: 0.2246
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0843 - mae: 0.2095 - val_loss: 0.0482 - val_mae: 0.1629
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0320 - mae: 0.1386 - val_loss: 0.0299 - val_mae: 0.1255
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0235 - mae: 0.1178 - val_loss: 0.0234 - val_mae: 0.1086
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0166 - mae: 0.0992 - val_loss: 0.0196 - val_mae: 0.0972
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0132 - mae: 0.0858 - val_loss: 0.0175 - val_mae: 0.0903
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0122 - mae: 0.0823 - val_loss: 0.0151 - val_mae: 0.0849
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.00

### 5) Definição e compilação do modelo Keras

Propósito: definir a arquitetura da rede (camadas Dense, ativações), compilar com otimizador e função de perda.

Entradas: shapes de `X_train` e `y_train`.

Saídas: objeto `model` compilado pronto para treinar.

Dicas:
- Ajuste o número de neurônios e camadas com base na complexidade do problema.
- Use `model.summary()` para inspecionar parâmetros e confirmar shapes.


In [ ]:
# Avaliar o modelo (carregar melhor modelo salvo se existir)
import os
best_path = 'model/queda_objeto_best.keras'
if os.path.exists(best_path):
    from tensorflow.keras.models import load_model
    model = load_model(best_path)
    print('Carregado melhor modelo salvo em', best_path)

loss, mae = model.evaluate(X_test, y_test)
print(f'Mean Absolute Error on test data: {mae:.6f}')

# Modelo final já salvo na etapa de treino como model/queda_objeto.keras

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0011 - mae: 0.0212  
Mean Absolute Error on test data: 0.021667804569005966


### 6) Treinamento com callbacks e salvamento

Propósito: treinar a rede com `ModelCheckpoint` e `EarlyStopping`, salvar o melhor modelo e o modelo final.

Entradas: `model`, `X_train`, `y_train`, hiperparâmetros de treino (epochs, batch_size).

Saídas: histórico de treino (loss/val_loss), arquivos `model/queda_objeto_best.keras` e `model/queda_objeto.keras`.

Dicas:
- Ajuste `validation_split` ou use um conjunto `X_val` separado para validação.
- Salve logs (TensorBoard) para análise se necessário.


In [ ]:
# Célula de inferência — carregar modelo e scalers e testar uma entrada
import numpy as np
import os
from tensorflow.keras.models import load_model
import joblib

model_path = 'model/queda_objeto.keras'
if not os.path.exists(model_path):
    raise SystemExit('Modelo não encontrado: ' + model_path + ' - execute a célula de treino primeiro')
model = load_model(model_path)
scaler_X = joblib.load('model/scaler_X.pkl')
scaler_y = joblib.load('model/scaler_y.pkl')

# Exemplo de entrada e previsão
m_val = 5.0
k_val = 0.5
t_val = 5.0

# Preparar entrada preservando nomes de features quando o scaler os tem
try:
    import pandas as pd
    if hasattr(scaler_X, 'feature_names_in_'):
        cols = list(scaler_X.feature_names_in_)
        df_input = pd.DataFrame([[m_val, k_val, t_val]], columns=cols)
        entrada_pad = scaler_X.transform(df_input)
    else:
        entrada = np.array([[m_val, k_val, t_val]])
        entrada_pad = scaler_X.transform(entrada)
except Exception as e:
    # fallback simples se pandas não estiver disponível ou ocorrer erro
    entrada = np.array([[m_val, k_val, t_val]])
    entrada_pad = scaler_X.transform(entrada)

saida_pad = model.predict(entrada_pad)
# Para scaler_y, verificar se espera colunas nomeadas também
try:
    # scaler_y geralmente não tem feature_names_in_, mas mantemos o fluxo
    saida = scaler_y.inverse_transform(saida_pad)
except Exception:
    saida = scaler_y.inverse_transform(saida_pad)

v_pred, y_pred = saida[0]
print(f'Previsão da rede — velocidade: {v_pred:.4f} m/s, posição: {y_pred:.4f} m')

# comparar com solução analítica
v_analitica = (m_val * 9.81 / k_val) * (1 - np.exp(-k_val * t_val / m_val))
y_analitica = (m_val * 9.81 / k_val) * t_val - (m_val**2 * 9.81 / k_val**2) * (1 - np.exp(-k_val * t_val / m_val))
print(f'Analítica — velocidade: {v_analitica:.4f} m/s, posição: {y_analitica:.4f} m')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Resultados Formula: Velocidade = 38.5993 m/s, Posição = 104.5066 m
Previsões da Rede Neural: Velocidade = 38.4627 m/s, Posição = 102.8974 m


c:\GitHub\ProjetosPessoais\redeNeuralArtificial\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


### 7) Inferência — carregar modelo e scalers, fazer previsão

Propósito: demonstrar como carregar o modelo salvo e os scalers e fazer previsões para entradas novas.

Entradas: valores de teste `m_val`, `k_val`, `t_val` (ou um arquivo CSV para batch). Também usa `model/queda_objeto.keras` e `model/scaler_*.pkl`.

Saídas: previsões de velocidade e posição para o tempo solicitado; comparação com a solução analítica quando aplicável.

Notas importantes:
- A célula verifica se `scaler_X.feature_names_in_` existe e constrói um `DataFrame` com as colunas esperadas para evitar o warning do sklearn.
- Se o modelo/scalers não existirem, a célula aborta com mensagem clara explicando como gerar/salvar os artefatos.


## Nota: por que ajustamos os scalers com DataFrames com nomes de coluna

O `StandardScaler` do scikit-learn, quando é ajustado (`fit`) usando um `pandas.DataFrame` com nomes de colunas, guarda esses nomes no atributo `feature_names_in_`.

Se você depois chamar `scaler.transform()` com um `numpy.ndarray` (sem nomes), o scikit-learn emite o warning:

```
UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
```

Para evitar esse warning e garantir que as colunas estejam na ordem correta durante a inferência, a célula de inferência abaixo verifica se `scaler_X.feature_names_in_` existe. Se existir, a entrada de teste é construída como um `DataFrame` com as mesmas colunas e ordem. Caso contrário, usa-se `ndarray` normal.

Essa prática garante reprodutibilidade entre treino e inferência e evita problemas sutis quando a ordem de features muda.

(Se preferir, podemos padronizar o pipeline salvando e reaplicando um `ColumnTransformer`/`Pipeline` do scikit-learn em vez de usar scalers independentes.)


### 8) Pequeno teste / utilitário (célula final)

Propósito: executar um teste rápido, plotar resultados ou fornecer utilitário para salvar/baixar artefatos.

Entradas: dependem do conteúdo da célula (p.ex. um CSV de entrada ou parâmetros de teste).

Saídas: gráficos, mensagens de status, arquivos salvos.

Dica: execute apenas após treinar e salvar o modelo; usar essa célula para gerar gráficos de comparação entre predição e verdade.
